# Mars Seismic Data Visualization


This is a notebook to download and visualize Mars seismic event data from the xxxx 3-component seismiometer

Here is a great readme on downloading the data and formatting it from NASA: https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/readme.txt

First need to:
 -  Choose the events by downloading the catalog, remove events that have default lat/long (4.5024, 135.6234)
 -  Download the raw data automatically from NASA (only horizontal component currently (BHU)
 -  Aggregate the data and compute differentiation
 -  Plot the events in panels

In [33]:
import pandas as pd
import numpy as np
import os
import urllib.request
import matplotlib.pyplot as plt
import requests
from io import StringIO

# place to save data
basedir = "C:/home/data/mars/"


def getEvents(starttime=None,endtime=None):
    url = "https://service.iris.edu/irisws/mars-event/1/query?minmagnitude=0&format=text"
    response = requests.get(url)
    plain_text = response.text
    lines = plain_text.strip().split('\n')
    data = [line.split('|') for line in lines]
    df = pd.DataFrame(data[1:], columns=data[0])
    df = df[(df['Latitude']!="4.5024") & (df['Latitude']!="135.6234")]
    df['Sol'] = df['ContributorID'].str[1:4]
    df['Time'] = pd.to_datetime(df['Time'])
    df['Year'] = df['Time'].dt.year.astype(str)
    df['DOY'] = (df['Time'].dt.dayofyear).astype(str).str.zfill(3)
    if starttime:
        df = df[df['Time']>starttime]
    if endtime:
        df = df[df['Time']<endtime]
    return df

events = getEvents()

# Download the seismicity data test (need come up with a list for event format)
# url_base = "https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/data/xb/continuous_waveform/elyse/2021/122/"
# file_name = "xb.elyse.02.bhu.2021.122.7.a.csv"
# url = url_base+file_name

# Time the Insight lander landed on 
lander_time = pd.to_datetime("2018-11-26T11:52:59.0Z")

def download_raw_data(url,file_name,save = False):
    d = pd.read_csv(url,skiprows=19, delimiter=',')
    # Option to save data
    if save:
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        d.to_csv(basedir+file_name,index=False)
    return d

def download_all_events(allevents):
    for indx,event in allevents.iterrows():
        doy = event['DOY']
        year = event['Year']
        url_base = f"https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/data/xb/continuous_waveform/elyse/{year}/{doy}/"
        for ix in range(15):
            try:
                # Only horizontal component being grabbed
                file_name = f"xb.elyse.02.bhu.{year}.{doy}.{ix}.a.csv"
                url = url_base+file_name
                download_raw_data(url,file_name,save=True)
                print(f'downloaded {file_name}')
                break
            except:
                continue

def read_downloaded_events(levents=None):
    dlist = []
    for file in os.listdir(basedir):
        sub = pd.read_csv(basedir+file).drop(columns=['Unnamed: 0'],errors='ignore').rename(columns={" Sample":"Sample"})
        sub['name'] = os.path.splitext(os.path.basename(file))[0]
        print('reading:',os.path.splitext(os.path.basename(file))[0])
        dlist.append(sub)
    return dlist
                

# Uncomment to download all events to disk in base_dir
#download_all_events(events)

dlist = read_downloaded_events()

rdata = pd.concat(dlist)
rdata['Time'] = pd.to_datetime(rdata['Time'])


print('done')

reading: xb.elyse.02.bhu.2021.122.7.a
reading: xb.elyse.02.bhu.2022.002.2.a
reading: xb.elyse.02.bhu.2022.034.7.a
reading: xb.elyse.02.bhu.2022.124.3.a
done


In [36]:
def decimate_and_filter(df, col_name, time_col_name, name_col_name, original_interval, desired_interval, window_size):
    unique_names = df[name_col_name].unique()

    result_dfs = []

    for name in unique_names:
        # Filter the input DataFrame by the unique name
        filtered_df = df[df[name_col_name] == name]

        # Get the original data from the filtered DataFrame
        time_series_data = filtered_df[col_name].values
        time_series_time = filtered_df[time_col_name].values

        # Define the original and desired sampling rates (in Hz)
        original_frequency = 1 / original_interval
        desired_frequency = 1 / desired_interval

        # Compute the decimation factor
        decimation_factor = int(original_frequency * desired_interval)

        # Define the sinc interpolation function
        def sinc_interp(x, s, factor):
            sinc = lambda x: np.sinc(x / factor)
            return np.dot(s, sinc(np.arange(len(s)) - x))

        # Perform the sinc interpolation and decimation
        decimated_data = []
        decimated_time = []
        for i in range(0, len(time_series_data), decimation_factor):
            decimated_data.append(sinc_interp(i, time_series_data[i:i+decimation_factor], decimation_factor))
            decimated_time.append(time_series_time[i])

        decimated_data = np.array(decimated_data)
        decimated_time = np.array(decimated_time)

        # Create a pandas DataFrame with the decimated data and time
        decimated_df = pd.DataFrame({time_col_name: decimated_time, col_name: decimated_data, name_col_name: [name] * len(decimated_data)})

        # Apply a rolling mean filter with the desired window size
        #decimated_df[f'{col_name}_rolling_mean'] = decimated_df[col_name].rolling(window=window_size).mean()

        result_dfs.append(decimated_df)

    # Concatenate the result DataFrames
    result_df = pd.concat(result_dfs, ignore_index=True)

    return result_df

def add_normalized_time_index(df, time_col_name, name_col_name):
    unique_names = df[name_col_name].unique()
    result_dfs = []

    for name in unique_names:
        # Filter the input DataFrame by the unique name
        filtered_df = df[df[name_col_name] == name].copy()

        # Calculate the total duration of the signal
        total_duration = filtered_df[time_col_name].max() - filtered_df[time_col_name].min()

        # Calculate the time elapsed since the start of the signal
        filtered_df['Time_elapsed'] = filtered_df[time_col_name] - filtered_df[time_col_name].min()

        # Normalize the time index
        filtered_df['Norm_Time'] = filtered_df['Time_elapsed'] / total_duration

        result_dfs.append(filtered_df)

    # Concatenate the result DataFrames
    result_df = pd.concat(result_dfs, ignore_index=True)

    return result_df

def process_data(data):
    data['Time'] = pd.to_datetime(data['Time'])
    data['Amplitude_Normalized'] = (data['Sample']-min(data['Sample']))/(max(data['Sample'])-min(data['Sample']))
    data['Amplitude_Diff'] = data['Sample'].diff()
    return data

# Decimate the data to 1-minute resolution and apply a rolling mean filter with a window size of 5
original_interval = 0.05  # 50 ms
desired_interval = 1  # 1 second
window_size = 5
#data = decimate_and_filter(rdata, 'Sample', 'Time', 'name', original_interval, desired_interval, window_size)
data = add_normalized_time_index(rdata, 'Time', 'name')

data = process_data(data)
display(data)

,Time,Sample,name,Time_elapsed,Norm_Time,Amplitude_Normalized,Amplitude_Diff
0,2021-05-02 00:00:00.016000+00:00,-4178,xb.elyse.02.bhu.2021.122.7.a,0 days 00:00:00,0.000000e+00,0.492981,NaN
1,2021-05-02 00:00:00.066000+00:00,-4206,xb.elyse.02.bhu.2021.122.7.a,0 days 00:00:00.050000,5.787060e-07,0.492973,-28.0
2,2021-05-02 00:00:00.116000+00:00,-4202,xb.elyse.02.bhu.2021.122.7.a,0 days 00:00:00.100000,1.157412e-06,0.492974,4.0
3,2021-05-02 00:00:00.166000+00:00,-4207,xb.elyse.02.bhu.2021.122.7.a,0 days 00:00:00.150000,1.736118e-06,0.492972,-5.0
4,2021-05-02 00:00:00.216000+00:00,-4272,xb.elyse.02.bhu.2021.122.7.a,0 days 00:00:00.200000,2.314824e-06,0.492952,-65.0
...,...,...,...,...,...,...,...
6911984,2022-05-04 23:59:59.662000+00:00,-4776,xb.elyse.02.bhu.2022.124.3.a,0 days 23:59:59.650000,9.999977e-01,0.492792,426.0
6911985,2022-05-04 23:59:59.712000+00:00,-4563,xb.elyse.02.bhu.2022.124.3.a,0 days 23:59:59.700000,9.999983e-01,0.492859,213.0
6911986,2022-05-04 23:59:59.762000+00:00,-3126,xb.elyse.02.bhu.2022.124.3.a,0 days 23:59:59.750000,9.999988e-01,0.493315,1437.0
6911987,2022-05-04 23:59:59.812000+00:00,-878,xb.elyse.02.bhu.2022.124.3.a,0 days 23:59:59.800000,9.999994e-01,0.494029,2248.0


In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go
import plotly

def plotEventPanels(data,yax="Amplitude_Diff",xax="Time"):
    # Get the unique event names
    event_names = data['name'].unique()

    # Create subplots with a shared x-axis
    fig = sp.make_subplots(rows=len(event_names), cols=1, shared_xaxes=True, subplot_titles=event_names, vertical_spacing=0.05)

    # Loop through event names and add a trace for each event
    for idx, event_name in enumerate(event_names):
        event_data = data[data['name'] == event_name]

        trace = go.Scatter(
            x=event_data[xax],
            y=event_data[yax],
            mode="lines",
            name=event_name,
            line=dict(width=2),
        )

        fig.add_trace(trace, row=idx + 1, col=1)

    # Update the layout with axis labels and a title
    fig.update_layout(
        title="Seismic Data from InSight SEIS",
        xaxis=dict(title="Date Time"),
        yaxis=dict(title="Amplitude Normalized"),
    )

    # Display the interactive plot
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig)

# Set column names
yax = "Amplitude_Diff"
xax = "Norm_Time"
plotEventPanels(data,yax=yax,xax=xax)


In [19]:
# Compute Amplitude Spectrums and plot for each event

# Set column names
yax = "Sample"
xax = "Time"

# Get the unique event names
event_names = data['name'].unique()

# Initialize an empty dictionary to store the FFT results
fft_results = {}

# Loop through event names and calculate the FFT for each event
for event_name in event_names:
    event_data = data[data['name'] == event_name]
    
    # Extract the time series and normalize it if necessary
    time_series = event_data[yax].values
    
    # Perform the FFT
    fft_output = np.fft.fft(time_series)
    
    # Store the FFT output in the results dictionary
    fft_results[event_name] = fft_output



# Calculate the number of data points in the time series
n = len(data[xax].values)

# Calculate the frequency values corresponding to the FFT output
freqs = np.fft.fftfreq(n)

# Create subplots with a shared x-axis
fig = sp.make_subplots(rows=len(event_names), cols=1, shared_xaxes=True, subplot_titles=event_names, vertical_spacing=0.05)

# Loop through event names and add a trace for each event's amplitude spectrum
for idx, event_name in enumerate(event_names):
    # Retrieve the FFT output from the results dictionary
    fft_output = fft_results[event_name]
    
    # Calculate the amplitude spectrum
    amplitude_spectrum = np.abs(fft_output)
    
    trace = go.Scatter(
        x=freqs,
        y=amplitude_spectrum,
        mode="lines",
        name=event_name,
        line=dict(width=2),
    )
    
    fig.add_trace(trace, row=idx + 1, col=1)

# Update the layout with axis labels and a title
fig.update_layout(
    title="Amplitude Spectrum of Seismic Data from InSight SEIS",
    xaxis=dict(title="Frequency (Hz)"),
    yaxis=dict(title="Amplitude"),
)

# Display the interactive plot
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)
